In [7]:
!pip install tensorflow

In [8]:
# 데이터 로드
from tensorflow import keras
# from tensorflow_core.python.keras.api import keras

# 훈련 데이터 테스트 데이터 분리해서 가져온다
(train_input, train_target), (test_input, test_target) = (
    keras.datasets.fashion_mnist.load_data()
)

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28 * 28)

#  데이터 정규화  인풋 나누기 255 
# 정규화 과정 (배치크기 , rows , cols )- > 평탄 
# -1 = 전체 설정 / 

# 예제   (32,28,28,3): 32개의 /28×28/   28×28 크기의 RGB 이미지를 배치로 처리.
# + 해석 image = np.random.randint(0, 255, (28, 28, 3)) = 0 = 최소 255 = 최댓값   (28, 28, 3) 배열

# 데이터셋 검증 세트 분리
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)


In [9]:
# # 리스트 형태로 밀집층 생성
# model = keras.Sequential(
#     [
#         keras.layers.Input(shape=(784,)),
#         keras.layers.Dense(
#             10, activation="softmax"
#         ),  # 활성화 함수: 다중 분류 모델은 softmax 함수 사용, 이진 분류에는 sigmoid 함수 사용
#     ]
# )  # ([])리스트 형태로 내보내기

# 객체 생성 후 레이어 추가 방식 
# model = keras.Sequential()  # 밀집층을 가진 신경망 모델 생성
# model.add(keras.layers.Input(shape=(784,)))
# model.add(keras.layers.Dense(10, activation="softmax"))

#  위 2개의 방식을 사용해서 신경망 설정 하면됨 

# 인공 신경망 모델에 층을 2개 추가 - 그림 참조 교재 368p
# 입력층과 출력츠 사이에 밀집층이 추가됐다.
# 이와 같이 입력층과 출력층 사이에 있는 모든 층은 은닉층이라 한다.
# 은닉층에는 활성화 함수가 있다. 이는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수다.
# 앞서 적용했던 소프트맥스 함수도 활성화 함수다.
# 출력층에 적용하는 활성화 함수는 종류가 제한되어 있다.
# 이진 분류일 경우 시그모이드 함수이고, 다중 분류일 경우 소프트맥스 함수를 사용한다.
# 이에 비해 은낵층의 활성화 함수는 비교적 자유롭다.
# 대표적으로 시그모이드 함수와 렐루 함수 등이 사용된다.

# 변수 전달 방식으로 시그모이드를 사용해 은닉층을, 소프트맥스를 사용해 출력층의 Dense 생성 

input_layer = keras.layers.Input(shape=(784,))
dense1 = keras.layers.Dense(100, activation='sigmoid')# 은닉층
dense2 = keras.layers.Dense(10, activation='softmax')# 출력층

# 제약 사항은 적어도 출력층의 뉴런이 적어야 한다 

# 심층 신경망 만들기 

model = keras.Sequential(
    [
    input_layer,
    dense1,
    dense2
    ]
)

# 출력층은 항상 맨 아래 위치한다. 
# 중간에 추가되면 은닉층은 증가한다. 

# 케라스 모델 요약 
model.summary()

# 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력된다.
# 출력 크기를 보면 (None, 100)이다. 첫번째 차원은 샘플의 개수다. 아직 정의되지 않아 None으로 출력됐다
# 캐라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행한다.
# 이를 미니 배치 하강법이라 한다.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# 렐루 함수
# 일반적으로 시그모이드 함수는 단점이 있다. 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누어있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다는 것이다.
# 특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 어렵게 만든다
# 이를 개선하기 위해 다른 종류의 활성화 함수가 제안됐는데, 이를 렐루 함수라 한다.


# 렐루 함수는 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수일 경우에는 0으로 만든다. 그림참조: 교재 377p
# 렐루 함수는 max(0, z)와 같이 사용한다.
# z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력한다.
# 이 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져 있다.


# 우리 데이터는 28*28 크기의 이미지이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape 메서드를 사용해 1차원으로 펼친다
# 캐라스의 Flatten을 제공한다
# Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼친다
# 입력에 곱해지는 가중치나 절편이 없다.
# 따라서 인공 신경망의 성능을 위해 기여하는 바가 없다.
# 하지만 Flatten 클래스를 층처럼 입력층과 은익층 사이에 추가하기 때문에 이를 층이라 부를 수 있다.
# Sequential 모델 생성 
model = keras.Sequential()

# Flatten layer 추가 

model.add(keras.layers.Flatten(input_shape=(28,28)))

#  Flatten layer 설명 : 2D를 1D 백터로 변환    
# 쓰는이유: flatten 레이어가 2차원을 인식함 -> 첫번쨰 레이어로써 필요함(인식단계)
# Dense Layer는 1D 형태의 데이터 만 처리 가능함  


# Dense Layer 
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()


C:\Anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
(train_input, train_target), (test_input, test_target) = (
    keras.datasets.fashion_mnist.load_data()
)

train_scaled = train_input / 255.0
# train_scaled = train_scaled.reshape(-1, 28 * 28) 
# flatten 층을 추가해서 할필요 없음 (keras는 추상화로 자동계산)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

# 모델 컴파일 후 훈련 
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7679 - loss: 0.6671
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8552 - loss: 0.4031
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8708 - loss: 0.3567
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8805 - loss: 0.3277
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8841 - loss: 0.3143


In [12]:
# 렐루 함수를 적용한 모델에 테스트 검증 세트 확인 
model.evaluate(val_scaled,val_target)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8735 - loss: 0.3625


[0.3636697232723236, 0.8739166855812073]